In [ ]:
from pprint import pprint
import time
from datetime import datetime
from dateutil import tz
import pytz
import json
from uuid import UUID
import asyncio
from copy import deepcopy
from vindicator.request.request_wynncraft import RequestWynncraft
from vindicator.database.vindicator_database import VindicatorDatabase

from typing import TYPE_CHECKING, Any, Dict, List

if TYPE_CHECKING:
    from aiohttp import ClientResponse

In [ ]:
def get_json(filename: str) -> Any:
    with open(filename, "r") as f:
        return json.load(f)

uuid_usernames_timestamp = get_json("uuid-username-timestamp.json")
asyncio.create_task(VindicatorDatabase.ainit())


In [ ]:
class Main:

    def __init__(self, uuid_usernames: List[Dict[str, str]]) -> None:
        self.uuid_usernames: List[Dict[str, str]] = uuid_usernames

    async def main(self):
        print(f"Attempting to insert {len(self.uuid_usernames)} username-uuid pairs to database")
        self.parse_uuid()

        print(type(self.uuid_usernames[0]["uuid"]))
        await self.to_db()

    def parse_uuid(self):
        for uuid_username in self.uuid_usernames:
            uuid_username["uuid"] = UUID(uuid_username["uuid"]).bytes

    async def to_db(self):
        t0 = time.perf_counter()
        query = (
            "INSERT INTO player_main_info (uuid, latest_username, timestamp) "
            "VALUES (%(uuid)s, %(latest_username)s, FROM_UNIXTIME(%(timestamp)s))"
        )
        print(f"Attempting to insert {len(self.uuid_usernames)} username-uuid pairs into database")
        await VindicatorDatabase.write_many(query, self.uuid_usernames)
        print(f"Inserted username-uuid pairs into database. Duration={time.perf_counter() - t0:.2f}s")


In [ ]:
main = Main(deepcopy(uuid_usernames_timestamp))
main_task = asyncio.create_task(main.main())

In [ ]:
print(main_task.done())
print(main_task.result())

In [ ]:
main_task.cancel()

In [ ]:
# async def main():
#     await VindicatorDatabase.ainit()
#     tables = await VindicatorDatabase.read_all(query="SHOW TABLES IN player_data;")
#     return tables

# tables = asyncio.gather(main())

In [ ]:
# for e in tables.result()[0]:
#     print(f"{e['Tables_in_player_data'].upper()} = \"{e['Tables_in_player_data']}\"")

In [ ]:
req_wynn = RequestWynncraft()
mojang_api = API()
asyncio.create_task(req_wynn.ainit())

In [ ]:
type(req_wynn.session._client_session)

In [ ]:
online_players = asyncio.create_task(req_wynn.get_online_player_list())
online_players_client_resp = asyncio.create_task(req_wynn.get_online_player_list(json_response=False))
current_timestamp = int(time())
# avicia = asyncio.create_task(req_wynn.get_guild_stats("Avicia", False))
# lindafelix = asyncio.create_task(req_wynn.get_player_stats("lindafelix"))
# fazuh = asyncio.create_task(req_wynn.get_player_stats("FAZuH"))
# marcus = asyncio.create_task(req_wynn.get_player_stats("Maarcus1"))

In [ ]:
pprint(pytz.all_timezones)

In [ ]:
r: "ClientResponse" = online_players_client_resp.result()

dates_request = datetime.strftime(datetime.fromtimestamp(current_timestamp), "%a, %d %b %Y %H:%M:%S %Z")
dates_response = r.headers["Date"][:-4]

print(f"{dates_request=}")
print(f"{dates_response=}")

ts_request = current_timestamp
ts_response = datetime.strptime(r.headers["Date"], "%a, %d %b %Y %H:%M:%S %Z").replace(tzinfo=tz.tzutc()).timestamp()

print()
print(f"{ts_request=}")
print(f"{ts_response=}")

print()
print(f"{ts_request - ts_response=}")


In [ ]:
online_players_content = online_players.result()
online_usernames = list(online_players_content['players'].keys())
# fazuh_content = fazuh.result()
# marcus_content = marcus.result()
# lindafelix_content = lindafelix.result()

In [ ]:
marcus_ch_uuids = [e for e in marcus_content["characters"].keys()]
lindafelix_ch_uuids = [e for e in lindafelix_content["characters"].keys()]

In [ ]:
xps = []
blockss = []
items_idd = []
skillpoints = []
raids = []
def thing(content: dict):
    xps = []
    blockss = []
    items_idd = []
    skillpoints = []
    raids = []
    ch_uuids = [e for e in content["characters"].keys()]
    for chuuid in ch_uuids:
        xps.append(content["characters"][chuuid]["xp"])
        blockss.append(content["characters"][chuuid]["blocksWalked"])
        items_idd.append(content["characters"][chuuid]["itemsIdentified"])
        skillpoints.append(content["characters"][chuuid]["skillPoints"])
        if content["characters"][chuuid]["raids"]:
            raids.append(content["characters"][chuuid]["raids"]["list"])
    return xps, blockss, items_idd, skillpoints, raids

In [ ]:
pprint(thing(marcus_content)[-1])

In [34]:
a = ["a_", "ab"]
print(sorted(a))

['a_', 'ab']
